# The mirroring is quite computational expensive therefore the diagonal blocks get precomputed

In [1]:
import cloudpickle
from scipy.sparse import lil_matrix,dok_matrix
from scipy.sparse import tril
import scipy
import re
import time

def load_dict(path,zeilen,spalten):
    if(zeilen >= spalten):
        template = "blockcounts_{i}_{j}".format(i=zeilen,j=spalten)
    else:
        template = "blockcounts_{i}_{j}".format(i=spalten,j=zeilen)

    file_path = path + '\\' + template
    co_occurences = {}
    
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            match = re.match('\(([0-9]{1,}), ([0-9]{1,})\):([0-9.]{1,})',line)
            x = int(match.group(1))
            y = int(match.group(2))
            count = float(match.group(3))
            if (x,y) in co_occurences:
                co_occurences[(x,y)] += count
            else:
                co_occurences[(x,y)] = count
    
    return co_occurences


def load_co_occurence(path,zeilen,spalten):
    co_occurences_dict = load_dict(path,zeilen,spalten)
    print(len(co_occurences_dict))
    coocurrence = dok_matrix((20000,20000),dtype='d')
    
    coocurrence._update(co_occurences_dict)
    
    coocurrence  = coocurrence.toarray()
    zero = tf.constant(0, dtype=tf.float32)
    where = tf.not_equal(coocurrence, zero)
    indices = tf.where(where)
    print(indices.shape[0])
    
    if spalten > zeilen :
        raise Exception("You sha'll not get symmetrical context")
        #may need to fix this in the future due to other non symmetrical feature
    
    if spalten == zeilen:
        print('mirroring')
        #print(coocurrence.toarray())
        coocurrence = coocurrence + tril(coocurrence,k=-1).transpose()
    return coocurrence

In [2]:
import re
import os
import h5py
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

# The default block length of 20000 must be divisible by this number. 
split_length = 4 #2,4,5,10
size = int(20000/split_length)

path = 'E:\\tmp\\coocurrence_blocks_nodyn'
path_hdf = r'E:\\tmp\\hdf_nodyn\\'
regex = r'blockcounts_([0-9]{1,})_([0-9]{1,})'

for file_name in os.listdir(path):
    match = re.match(regex, file_name)
    i,j   = match.group(1), match.group(2)
    i,j   = int(i)        , int(j)
    print('output for progress ' + str(i)+','+str(j))
    if i< j:
        print('skipping')
        continue
    

    co_occurence = load_co_occurence(path,i,j)
    if(i == j):
        np.fill_diagonal(co_occurence,0)
    
    for sub_i in range(split_length):
        for sub_j in range(split_length):
            a,b = i*split_length+sub_i , j*split_length+sub_j
            print(a,b)
            filename = 'tf_cooccurence_{a}_{b}.hdf'.format(a = a,b = b)
            
            f = h5py.File( path_hdf +filename, "w")#plus experiment name
            co_occurence_hdf = f.create_dataset("co-ocurrence", (size, size))
            part = co_occurence[sub_i*size:(sub_i+1)*size,sub_j*size:(sub_j+1)*size]
            co_occurence_hdf[:,:] = part
            f.close()
            
        
    #print(co_occurence_hdf)

output for progress 0,0
1222760
1222760
mirroring
0 0
0 1
0 2
0 3
1 0
1 1
1 2
1 3
2 0
2 1
2 2
2 3
3 0
3 1
3 2
3 3
output for progress 10,0
2355716
2355716
40 0
40 1
40 2
40 3
41 0
41 1
41 2
41 3
42 0
42 1
42 2
42 3
43 0
43 1
43 2
43 3
output for progress 10,1
2240581
2240581
40 4
40 5
40 6
40 7
41 4
41 5
41 6
41 7
42 4
42 5
42 6
42 7
43 4
43 5
43 6
43 7
output for progress 10,10
1164695
1164695
mirroring
40 40
40 41
40 42
40 43
41 40
41 41
41 42
41 43
42 40
42 41
42 42
42 43
43 40
43 41
43 42
43 43
output for progress 10,2
2216784
2216784
40 8
40 9
40 10
40 11
41 8
41 9
41 10
41 11
42 8
42 9
42 10
42 11
43 8
43 9
43 10
43 11
output for progress 10,3
2322149
2322149
40 12
40 13
40 14
40 15
41 12
41 13
41 14
41 15
42 12
42 13
42 14
42 15
43 12
43 13
43 14
43 15
output for progress 10,4
2417534
2417534
40 16
40 17
40 18
40 19
41 16
41 17
41 18
41 19
42 16
42 17
42 18
42 19
43 16
43 17
43 18
43 19
output for progress 10,5
2357396
2357396
40 20
40 21
40 22
40 23
41 20
41 21
41 22
41 23
42 2

25 21
25 22
25 23
26 20
26 21
26 22
26 23
27 20
27 21
27 22
27 23
output for progress 6,6
1231746
1231746
mirroring
24 24
24 25
24 26
24 27
25 24
25 25
25 26
25 27
26 24
26 25
26 26
26 27
27 24
27 25
27 26
27 27
output for progress 7,0
2346084
2346084
28 0
28 1
28 2
28 3
29 0
29 1
29 2
29 3
30 0
30 1
30 2
30 3
31 0
31 1
31 2
31 3
output for progress 7,1
2228650
2228650
28 4
28 5
28 6
28 7
29 4
29 5
29 6
29 7
30 4
30 5
30 6
30 7
31 4
31 5
31 6
31 7
output for progress 7,2
2204459
2204459
28 8
28 9
28 10
28 11
29 8
29 9
29 10
29 11
30 8
30 9
30 10
30 11
31 8
31 9
31 10
31 11
output for progress 7,3
2308378
2308378
28 12
28 13
28 14
28 15
29 12
29 13
29 14
29 15
30 12
30 13
30 14
30 15
31 12
31 13
31 14
31 15
output for progress 7,4
2408562
2408562
28 16
28 17
28 18
28 19
29 16
29 17
29 18
29 19
30 16
30 17
30 18
30 19
31 16
31 17
31 18
31 19
output for progress 7,5
2347072
2347072
28 20
28 21
28 22
28 23
29 20
29 21
29 22
29 23
30 20
30 21
30 22
30 23
31 20
31 21
31 22
31 23
output for p